# Weather data analysis


In [12]:
import pandas as pd
import json

First load the sample dataframe and extract each column info

In [3]:
df = pd.read_csv('data/prueba.csv')

In [ ]:
v_dict = {}
v_data = json.load(open('data/variables.json', encoding='UTF-8'))
for var in v_data:
    v_dict[var['codi_variable']] = var['acronim']

In [10]:
cols = list(df.columns)
cols[0] = 'id'


for i in cols[1:]:
    print(i)


['id', '72', '3', '30', '31', '32', '33', '34', '35', '36', '38', '40', '42', '44', '46', '47', '48', '49', '50', '51', '53', '54', '56', '57', '59', '1', '2']
72
3
30
31
32
33
34
35
36
38
40
42
44
46
47
48
49
50
51
53
54
56
57
59
1
2
